# Customize File handling with Callbacks.


All Sarracenia components implement *the Flow* algorithm, with different
callbacks, in the Python programming language. Sarracenia's main (Python) 
class is [sarracenia.flow](../Reference/code.html#module-sarracenia.flow) and the a great deal of core functionality is 
implemented using the class created to add custom processing to a flow, the 
flowcb (flow callback) class.

For a detailed discussion of the flow algorithm itself, have a look
at [Concepts](../Explanation/Concepts.html) manual. For any flow, one can
add custom processing at a variety of times during processing by sub-classing
the [sarracenia.flowcb](../Reference/flowcb.html) class.

Briefly, the algorithm has the following steps:

* **__init__(self, options)** -- when the import happens, traditional python initialization
* **on_start** -- when an instance is started.
* loop forever
   * **gather** -- collect messages to be processed called: worklist.incoming
   * **poll** -- another way to collect messages, only in the poll component.
   * **filter** -- apply accept/reject regular expression matches to the message list.
     moves messages for files not to download from worklist.incoming to worklist.reject
      * *after_accept* callback entry point. process worklist.incoming, potentially rejecting some more.
   * **ack** -- worklist.rejected messages are acknowledged to upstream source as processing is complete.
   * **work** -- perform a transfer or transformation on a file.
   * **ack** -- worklist.ok messages for successfully transferred files are acknowledged to upstream source.
      * *after_work* callback entry point
   * **ack** -- worklist.failed messages for files which not successfully transferred are acknowledged.
   * **post**  -- post the result of the work done for the next step.
   * occasionaly... **on_housekeeping  -- do periodic cleanups...
* **on_stop** -- shutdown processing.

for more details about flowcb entry points available, have a look at the source code: 

* [flowcb](../Reference/flowcb.html)


Lets look at using the class in a configuration:

In [1]:
!sr3 remove subscribe/hpfx_amis.conf
!sr3 add subscribe/hpfx_amis.conf

2022-03-19 13:19:27,886 2724830 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 13:19:27,887 2724830 [INFO] root remove removing subscribe/hpfx_amis 

add: 2022-03-19 13:19:28,372 2724832 [INFO] sarracenia.sr add copying: /home/peter/Sarracenia/sr3/sarracenia/examples/subscribe/hpfx_amis.conf to /home/peter/.config/sr3/subscribe/hpfx_amis.conf 



In [2]:
!echo messageCountMax 10 >>~/.config/sr3/subscribe/hpfx_amis.conf

have the flow stop after 10 messages are consumed.

In [3]:
!sr3 list fcb

2022-03-19 13:19:36,284 2724838 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
Provided callback classes: ( /home/peter/Sarracenia/sr3/sarracenia ) 
flowcb/accept/delete.py          flowcb/accept/downloadbaseurl.py 
flowcb/accept/hourtree.py        flowcb/accept/httptohttps.py     
flowcb/accept/longflow.py        flowcb/accept/posthourtree.py    
flowcb/accept/postoverride.py    flowcb/accept/printlag.py        
flowcb/accept/rename4jicc.py     flowcb/accept/renamedmf.py       
flowcb/accept/renamewhatfn.py    flowcb/accept/save.py            
flowcb/accept/speedo.py          flowcb/accept/sundewpxroute.py   
flowcb/accept/testretry.py       flowcb/accept/toclusters.py      
flowcb/accept/tohttp.py          flowcb/accept/tolocal.py         
flowcb/accept/tolocalfile.py     flowcb/accept/wmotypesuffix.py   
flowcb/clamav.py                 flowcb/filter/deleteflowfiles.py 
flowcb/filter/fdelay.py          flowcb/filter/pclean_f90

Adding that line to the configuration means that the wistree flowcb subclass (source above) will be added to 
the flow, changing processing by having its routines called... the main one being *after_accept*

In [4]:
!echo callback wistree >>~/.config/sr3/subscribe/hpfx_amis.conf

In [5]:
!sr3 foreground subscribe/hpfx_amis.conf

2022-03-19 13:19:48,073 2724872 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
.2022-03-19 13:19:48,285 [INFO] 2724874 sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 13:19:48,287 [INFO] 2724874 sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 13:19:48,287 [INFO] 2724874 sarracenia.flow loadCallbacks plugins to load: ['sarracenia.flowcb.gather.message.Message', 'sarracenia.flowcb.retry.Retry', 'sarracenia.flowcb.housekeeping.resources.Resources', 'sarracenia.flowcb.wistree.Wistree', 'sarracenia.flowcb.log.Log']
2022-03-19 13:19:48,690 [INFO] 2724874 sarracenia.flowcb.log __init__ subscribe initialized with: {'after_work', 'after_accept', 'on_housekeeping'}
2022-03-19 13:19:48,690 [INFO] 2724874 sarracenia.flow run options:
_Config__admin="...st/ None True True False False None None",
_Config__broker="...ca/ None 

Without the plugin, the download would put all files directly the reception directory. with the addition of the wistree callback, it puts places the file in /tmp/hpfx_amis.  With the change it puts it in the WIS tree of directories, and adds a file type suffix.


## Config File Entries and Callbacks


[flowcb.log](../Reference/flowcb.html#module-sarracenia.flowcb.log)

To add a callback to a a flow, a line is added to the flows's configuration file:

    flowcb sarracenia.flowcb.log.Log

If you follow the convention, and the name of the class is a capitalized
version (Log) of the file name (log), then a shorthand is available:

    callback log 

Either way it is done, it will cause Sarracenia to import the class and then
look for entry points in the class to call at appropriate times.

The class constructor accepts a sarracenia.config.Config class object,
called options, that stores all the settings to be used by the running flow.
Options is used to override default behaviour of both flows and callbacks.
The argument to the flowcb is a standard python class that needs to be
in the normal python path for python *import*, and the last element
is the name of the class in within the file that needs to be instantiated
as a flowcb instance.

a setting for a callback is declared as follows:

    set sarracenia.flowcb.filter.log.Log.logLevel debug

(the prefix for the setting matches the type hierarchy in flowCallback)

when the constructor for the callback is called, it's options
argument will contain:

    options.logLevel = 'debug'

If no module specific override is present, then the more global
setting is used.

So usage of callbacks can be made without much python knowledge at all,
just the ability to create configuration files.

Beyond this point, we find advice for people who want to write their
own callbacks in Python. Callbacks are ordinary Python, with a few wrinkles:

## Writing Your Own Callbacks


A flow callback, is a python class built with routines named to
indicate when the programmer wants them to be called.
To do that, create a routine which subclasses *sarracenia.flowcb.FlowCB*
so the class will normally have:

   from sarracenia.flowcb import FlowCB

in among the imports near the top of the file.
In the main part of the file, there will be the
custom callback classes:

  class Myclass(FlowCB):

declared as a subclass as FlowCB. The main routines in the class  are entry points
that will be called at the time their name implies. If you a class is missing a
given entry point, it will just not be called. The __init__() class is used to
initialize things for the callback class:

    def __init__(self, options):

        self.o = options

        logging.basicConfig(format=self.o.logFormat,
                            level=getattr(logging, self.o.logLevel.upper()))
        logger.setLevel(getattr(logging, self.o.logLevel.upper()))

        self.o.add_option( 'myoption', 'str', 'usuallyThis')

The logging setup lines in __init__ allow setting a specific logging level
for this flowCallback class. Once the logging boiler-plate is done,
the add_option routine to define settings to for the class.
users can include them in configuration files, just like built-in options:

        myoption IsReallyNeeded

The result of such a setting is that the *self.o.myoption = 'IsReallyNeeded'*.
If no value is set in the configuration, *self.o.myoption* will default to *'usuallyThis'*
There are various *kinds* of options, where the declared type modifies the parsing:
           
    'count'    integer count type. 
    'duration' a floating point number indicating a quantity of seconds (0.001 is 1 milisecond)
               modified by a unit suffix ( m-minute, h-hour, w-week ) 
    'flag'     boolean (True/False) option.
    'list'     a list of string values, each succeeding occurrence catenates to the total.
               all v2 plugin options are declared of type list.
    'size'     integer size. Suffixes k, m, and g for kilo, mega, and giga (base 2) multipliers.
    'str'      an arbitrary string value, as will all of the above types, each 
               succeeding occurrence overrides the previous one.



## Worklists

Besides *options*, the other main argument to after_accept and after_work callback
routines is the worklist. The worklist is given to entry points occurring during message
processing, and is a number of worklists of messages:

    worklist.incoming --> messages to process (either new or retries.)
    worklist.ok       --> successfully processed
    worklist.rejected --> messages to not be further processed.
    worklist.failed   --> messages for which processing failed.
                          failed messages will be retried.
    worklist.directories_ok --> list of directories created during processing.

Initially, all messages are placed in worklists.incoming.
if a plugin decides:

- a message is not relevant, moved it to the rejected worklist.
- a no further processing of the message is needed, move it to ok worklist.
- an operation failed and it should be retried later, move to failed worklist.

Do not remove from all lists, only move messages between the worklists.
it is necessary to put rejected messages in the appropriate worklist
so that they are acknowledged as received. Messages can only removed
after the acknowledgement has been taken care of.

## Logging


Python has great built-in logging, and once has to just use the module
in a normal, pythonic way, with::

  import logging

After all imports in your python source file, then define a logger
for the source file::

  logger = logging.getLogger(\__name\__)

As is normal with the Python logging module, messages can then
be posted to the log::

  logger.debug('got here')

Each message in the log will be prefixed with the class and routine
emitting the log message, as well as the date/time.



## Sample Flowcb Sub-Class

With the above information about option handling, worklists, and logging, we
are ready to understand the wistree module we just used.
This wistree.py class accepts files whose
names begin with AHL's, and renames the directory tree to a different standard,
the evolving one for the WMO WIS 2.0 (for more information on that module:
https://github.com/wmo-im/GTStoWIS2)

In [6]:
  from sarracenia.flowcb import FlowCB
  import logging
  import GTStoWIS2

  logger = logging.getLogger(__name__)


  class Wistree(FlowCB):

    def __init__(self, options):

        if hasattr(options, 'logLevel'):
            logger.setLevel(getattr(logging, options.logLevel.upper()))
        else:
            logger.setLevel(logging.INFO)
        self.topic_builder=GTStoWIS2.GTStoWIS2()
        self.o = options


    def after_accept(self, worklist):

        new_incoming=[]

        for msg in worklist.incoming:

            # fix file name suffix.
            type_suffix = self.topic_builder.mapAHLtoExtension( msg['new_file'][0:2] )
            tpfx=msg['subtopic']
    
            # input has relpath=/YYYYMMDDTHHMM/... + pubTime
            # need to move the date from relPath to BaseDir, adding the T hour from pubTime.
            try:
                new_baseSubDir=tpfx[0]+msg['pubTime'][8:11]
                t='.'.join(tpfx[0:2])+'.'+new_baseSubDir
                new_baseDir = msg['new_dir'] + os.sep + new_baseSubDir
                new_relDir = 'WIS' + os.sep + self.topic_builder.mapAHLtoTopic(msg['new_file'])
                new_dir = new_baseDir + os.sep + new_relDir
                
                if msg['new_file'][-len(type_suffix):] != type_suffix:      
                    new_file = msg['new_file']+type_suffix
                else:
                    new_file = msg['new_file']
                    
                msg.updatePaths( self.o, new_baseDir + os.sep + new_relDir, new_file )
            except Exception as ex:
                logger.error( "skipped" , exc_info=True )
                worklist.failed.append(msg)
                continue
    
            msg['_deleteOnPost'] |= set( [ 'from_cluster', 'sum', 'to_clusters' ] )
            new_incoming.append(msg)

        worklist.incoming=new_incoming 




## Plugins That Change How a File is Downloaded


The *after_accept* routine is one of the two most common ones in use. It is used to change processing prior to a file being downloaded or sent. To process the file after it has been downloaded, the *after_work* entry point is used to process the worklist.ok (files that were successfully downloaded) list.

The after_accept routine has an outer loop that cycles through the entire list of incoming messages. It builds a new list of incoming messages from the ones it accepts, while  appending all the rejected ones to *worklist.failed.* The list is just a list of messages, where each message is a python dictionary with all the fields stored in a v03 format message. In the message there are, for example, *baseURL* and *relPath* fields:

* baseURL - the baseURL of the resource from which a file would be obtained.
* relPath - the relative path to append to the baseURL to get the complete download URL.

This is happenning before transfer (download or sent, or processing) of the file has occurred, so one can change the behaviour by modifying fields in the message. Normally, the download paths (called new_dir, and new_file) will reflect the intent to mirror the original source tree. so if you have *a/b/c.txt*  on the source tree, and are downloading in to directory *mine* on the local system, the new_dir would be *mine/a/b* and new_file would be *c.txt*.

## Plugins that Process a file after it is Downloaded


A common use case is for plugins with an *after_work* entry point to read the file after it is downloaded and transform it into some derived product with a different name. So the new file is created as in the previous section. The message for the downloaded file still needs to be moved onto a list to ensure that it is acknowledged to the broker. Such an entry point would look like this:

In [9]:

    def after_work(self, worklist):

        new_ok=[]
        for m in worklist.ok:
             success=do_something()
             if success:
                   new_ok.append(m)
             # since it is already acknowledged, we can just drop it from ok.
             
             
        worklist.ok = new_ok
        # the messages on worklist.ok will get posted in the next algorithm phase.


## Plugins that Rename Files


The plugin above changes the layout of the files that are to be downloaded, based on the [GTStoWIS](https://github.com/wmo-im/GTStoWIS) class, which prescribes a different directory tree on output. There are a lot of fields to update when changing file placement, so best to use:

   msg.updatePaths( self.o, new_dir, new_file )

to update all necessary fields in the message properly. It will update 'new_baseURL', 'new_relPath', 'new_subtopic' for use when posting.

The try/except part of the routine deals with the case that, should a file arrive with a name from which a topic tree cannot be built, then a python exception may occur, and the message is added to the failed worklist, and will not be processed by later plugins.

## Plugins That Create New Files


The routine above is perfect when a file is just renamed. If a plugin needs to create new files only vaguely derived from the input file, then you want to create new messages for these files from scratch:

    import sarracenia

    m = sarracenia.Message.fromFileData(sample_fileName, self.o, os.stat(sample_fileName) )
   
The msg_fromFileData routine will use self.o to apply the appropriate posting settings.
no knowledge of message formats, or construction of fields is needed. If the file is not
local, such as when writing a poll callback, an alternate routing can be used:

    m = sarracenia.Message.fromFileInfo(sample_fileName, self.o, fake_stat_info )

the fake stat record (as per the stat(2) man page or python os.stat() ) can be built from other fields, starting with:

    import paramiko

    fake_stat = paramiko.SFTPAttributes()
    fake_stat.st_mtime = ... something else... perhaps an http header?
    fake_stat.st_size = ... again will vary by context.
    
Either way, once you have the message, it can be appended to the incoming list.


## Other Examples


Subclassing of [Sarracenia.flowcb](../Reference/flowcb.html) is used internally to do a lot of core work. It's a good idea to look at the sarracenia source code itself. For example:

* [sarracenia.flowcb](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/__init__.py) have a look at the __init__.py file in there, which
  provides this information on a more programmatically succinct format.

* [sarracenia.flowcb.gather.file](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/gather/file.py)
  is a class that implements
  file posting and directory watching, in the sense of a callback that
  implements the *gather* entry point, by reading a file system and building a
  list of messages for processing.

* [sarracenia.flowcb.gather.message](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/gather/message.py)
is a class that implements reception of messages from message queue protocol flows.

* [sarracenia.flowcb.gather.nodupe](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/nodupe)
This modules removes duplicates from message
  flows based on Integrity checksums.

* [sarracenia.flowcb.post.message](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/post/message.py)
is a class that implements posting
  messages to Message queue protocol flows

* [sarracenia.flowcb.retry](https://github.com/MetPX/Sarracenia/blob/v03_wip/sarracenia/flowcb/retry.py)
when the transfer of a file fails. Sarracenia needs to persist the relevant message to a state file for
  a later time when it can be tried again. 

